In [2]:
import requests
from bs4 import BeautifulSoup
import datetime

In [3]:
START_YEAR = 2009
END_YEAR = datetime.datetime.now().year - 1

In [7]:
# loop through all years
for i in range(START_YEAR, END_YEAR + 1):
    # find the last Monday of the year
    for j in range(31, 25, -1):
        # collect singles data
        url = f"https://www.atptour.com/en/rankings/singles?rankRange=1-5000&rankDate={i}-12-{j}"

        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        ranking_table = soup.find("table", {"class": "mega-table"})
        rows = ranking_table.find_all("tr")

        # skip if there is no data meaning that the date is not a Monday
        if len(rows) == 1:
            continue

        for row in rows[1:]:
            cells = row.find_all("td")

        # TODO: collect relevant data from cells and save to gzipped csv in data/overview/singles/{year}.gz
        

        # collect doubles data
        url = f"https://www.atptour.com/en/rankings/doubles?rankRange=1-5000&rankDate={i}-12-{j}"

        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        ranking_table = soup.find("table", {"class": "mega-table"})
        rows = ranking_table.find_all("tr")

        for row in rows[1:]:
            cells = row.find_all("td")

        # TODO: collect relevant data from cells and save to gzipped csv in data/overview/doubles/{year}.gz         
        
        break

In [1]:
# TODO: for each data file, loop through each player and collect their activity
# data for the give year and save to gzipped csv in data/activity/{singles or doubles}/{year}.gz
# only collect data for players inside the top 750 for singles and the top 250 for doubles

